## Selecting Rows with Core or ORM

For both Core and ORM, the `select()` function generates a `Select construct` which is used for all `SELECT` queries. Passed to methods like _`Connection.execute()` in Core_ and _`Session.execute()` in ORM_, a `SELECT statement` is emitted in the _current transaction_ and the result rows available via the returned `Result` object.

#### Initialize tables with some data

In [1]:
from sqlalchemy import (
    MetaData, Table, Column, Integer, String, ForeignKey, create_engine,
    insert, select, bindparam, func, cast, text, literal_column, and_, or_,
)
from sqlalchemy.orm import Session, registry, relationship

In [2]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
metadata_obj = MetaData()

In [3]:
user_table = Table(
    "user_account", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

address_table = Table(
    "address", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False)
)

metadata_obj.create_all(engine)

2022-09-26 13:03:52,169 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-26 13:03:52,172 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-09-26 13:03:52,173 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-26 13:03:52,174 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-09-26 13:03:52,175 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-26 13:03:52,178 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-09-26 13:03:52,179 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-26 13:03:52,180 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-09-26 13:03:52,181 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-26 13:03:52,183 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-09-26 13:03:52,184 INFO sqlalchemy.engine.Engine [no key 0.00094s] ()
2022-09-26 13:03:52,188 INFO sqlalchemy.engine.Engine 
C

In [4]:
stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"}
        ]
    )
    conn.commit()

2022-09-26 13:03:52,287 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-26 13:03:52,289 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-09-26 13:03:52,291 INFO sqlalchemy.engine.Engine [generated in 0.00367s] ('spongebob', 'Spongebob Squarepants')
2022-09-26 13:03:52,294 INFO sqlalchemy.engine.Engine COMMIT
2022-09-26 13:03:52,296 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-26 13:03:52,297 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-09-26 13:03:52,298 INFO sqlalchemy.engine.Engine [generated in 0.00175s] (('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star'))
2022-09-26 13:03:52,299 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
scalar_subq = (
    select(user_table.c.id).
    where(user_table.c.name == bindparam("username")).
    scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {"username": 'spongebob', "email_address": "spongebob@sqlalchemy.org"},
            {"username": 'sandy', "email_address": "sandy@sqlalchemy.org"},
            {"username": 'sandy', "email_address": "sandy@squirrelpower.org"},
        ]
    )
    conn.commit()

2022-09-26 13:03:52,385 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-26 13:03:52,387 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2022-09-26 13:03:52,388 INFO sqlalchemy.engine.Engine [generated in 0.00332s] (('spongebob', 'spongebob@sqlalchemy.org'), ('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org'))
2022-09-26 13:03:52,391 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
mapped_registry = registry()
Base = mapped_registry.generate_base()

In [7]:
class User(Base):
    __tablename__ = "user_account"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    
    addresses = relationship("Address", back_populates="user")
    
    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

In [8]:
class Address(Base):
    __tablename__ = "address"
    
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"))
    
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [9]:
mapped_registry.metadata.create_all(engine)

2022-09-26 13:03:52,893 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-26 13:03:52,895 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-09-26 13:03:52,896 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-26 13:03:52,898 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-09-26 13:03:52,899 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-26 13:03:52,901 INFO sqlalchemy.engine.Engine COMMIT


#### The `select()` SQL Expression Construct

The `select()` construct builds up a statement in the same way as that of `insert()`, using a _generative approach_ where each method builds more state onto the object. Like the other SQL constructs, it can be _stringified_ in place.

In [10]:
stmt = select(user_table).where(user_table.c.name == "spongebob")
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


Also in the same manner as all other statement-level `SQL constructs`, _to actually run the statement_ we pass it to an `execution` method. Since a `SELECT statement` returns _rows_ we can always iterate the result object to get _Row_ objects back.

In [11]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2022-09-26 13:03:53,140 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-26 13:03:53,142 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-09-26 13:03:53,144 INFO sqlalchemy.engine.Engine [generated in 0.00335s] ('spongebob',)
(1, 'spongebob', 'Spongebob Squarepants')
2022-09-26 13:03:53,146 INFO sqlalchemy.engine.Engine ROLLBACK


When using the ORM, particularly with a `select()` construct that's _composed against ORM entities_, we will want to execute it using the `Session.execute()` method on the `Session`; using this approach, we continue to get `Row` objects from the result, however these rows are now _capable of including **complete entities**_, such as instances of the `User` class, as individual elements within each row.

In [12]:
stmt = select(User).where(User.name == "spongebob")
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2022-09-26 13:03:53,240 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-26 13:03:53,247 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-09-26 13:03:53,248 INFO sqlalchemy.engine.Engine [generated in 0.00192s] ('spongebob',)
(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)
2022-09-26 13:03:53,250 INFO sqlalchemy.engine.Engine ROLLBACK


While the SQL generated in these examples looks the same whether we invoke `select(user_table)` or `select(User)`, in the more general case they _do not necessarily render the same thing_, as an ORM-mapped class may be mapped to other kinds of `"selectables"` besides tables. __The `select()` that's against an ORM entity also indicates that ORM-mapped instances should be returned in a result, which is not the case when SELECTing from a Table object.__

#### Setting the COLUMNS and FROM clause

The `select()` function accepts positional elements representing any number of `Column` and/or `Table` expressions, as well as a _wide range of compatible objects_, which are resolved into a __list of SQL expressions__ to be `SELECT`ed from that will be returned as columns in the result set. These elements also serve in simpler cases to create the `FROM` clause, which is inferred from the _columns and table-like expressions_ passed.

In [13]:
print(select(user_table))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


To `SELECT` from individual columns using a Core approach, `Column` objects are accessed from the _`Table.c` accessor_ and __can be sent directly__; the `FROM` clause will be inferred as the set of all `Table` and other `FromClause` objects that are represented by those columns.

In [14]:
print(select(user_table.c.name, user_table.c.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


##### Selecting ORM Entities and Columns

_ORM entities_, such our `User class` as well as the _column-mapped attributes upon it_ such as `User.name`, also participate in the `SQL Expression Language system` representing __tables and columns__. Below illustrates an example of `SELECT`ing from the `User` entity, which ultimately renders in the same way as if we had used `user_table` directly.

In [15]:
print(select(User))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


When executing a statement like the above using the _ORM `Session.execute()` method_, there is __an important difference__ when we _select from a full entity such as `User`, as opposed to `user_table`_, which is that the `entity` itself is returned as _a single element_ within each row. That is, when we fetch rows from the above statement, as there is only the User entity in the list of things to fetch, we get back `Row` objects that have only one element, which contain _instances of the `User` class_.

In [16]:
row = session.execute(select(User)).first()
print(row)

2022-09-26 13:03:53,628 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-26 13:03:53,631 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2022-09-26 13:03:53,633 INFO sqlalchemy.engine.Engine [generated in 0.00118s] ()
(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)


The above `Row` has just one element, representing the `User` entity.

In [17]:
print(row[0])

User(id=1, name='spongebob', fullname='Spongebob Squarepants')


A highly recommended convenience method of achieving the same result as above is to use the `Session.scalars()` method to _execute the statement **directly**_; this method will return a `ScalarResult` object that delivers the _first `column` of each row at once_, in this case, _instances of the `User` class_.

In [18]:
user = session.scalars(select(User)).first()
print(user)

2022-09-26 13:03:53,848 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2022-09-26 13:03:53,849 INFO sqlalchemy.engine.Engine [cached since 0.2181s ago] ()
User(id=1, name='spongebob', fullname='Spongebob Squarepants')


Alternatively, we can _select individual columns_ of an ORM entity as `distinct` elements within result rows, by _using the class-bound attributes_; when these are passed to a construct such as `select()`, they are resolved into the `Column` or other SQL expression represented by each attribute.

In [19]:
print(select(User.name, User.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


When we invoke this statement using `Session.execute()`, we now receive rows that have _individual elements per value_, each corresponding to a separate column or other SQL expression.

In [20]:
row = session.execute(select(User.name, User.fullname)).first()
print(row)

2022-09-26 13:03:54,055 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account
2022-09-26 13:03:54,056 INFO sqlalchemy.engine.Engine [generated in 0.00117s] ()
('spongebob', 'Spongebob Squarepants')


The approaches can also be mixed, as below where we `SELECT` the _name attribute_ of the User entity as the `first element of the row`, and **combine** it with _full Address entities_ in the `second element`.

In [21]:
session.execute(
    select(User.name, Address).
    where(User.id==Address.id).
    order_by(Address.id)
).all()

2022-09-26 13:03:54,174 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.id, address.email_address, address.user_id 
FROM user_account, address 
WHERE user_account.id = address.id ORDER BY address.id
2022-09-26 13:03:54,175 INFO sqlalchemy.engine.Engine [generated in 0.00162s] ()


[('spongebob', Address(id=1, email_address='spongebob@sqlalchemy.org')),
 ('sandy', Address(id=2, email_address='sandy@sqlalchemy.org')),
 ('patrick', Address(id=3, email_address='sandy@squirrelpower.org'))]

Approaches towards _selecting ORM entities and columns_ as well as common methods for _converting rows_ are discussed further at `Selecting ORM Entities and Attributes`.

##### Selecting from Labeled SQL Expressions

The `ColumnElement.label()` method as well as the _same-named method_ available on `ORM attributes` provides a _SQL label of a column or expression_, allowing it to have a **specific name** in a result set. This can be helpful when _referring to arbitrary SQL expressions in a result row by name_.

In [22]:
stmt = (
    select(
        ("Username: " + user_table.c.name).label("username"),
    ).order_by(user_table.c.name)
)

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.username}")

2022-09-26 13:03:54,295 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-26 13:03:54,296 INFO sqlalchemy.engine.Engine SELECT ? || user_account.name AS username 
FROM user_account ORDER BY user_account.name
2022-09-26 13:03:54,297 INFO sqlalchemy.engine.Engine [generated in 0.00228s] ('Username: ',)
Username: patrick
Username: sandy
Username: spongebob
2022-09-26 13:03:54,300 INFO sqlalchemy.engine.Engine ROLLBACK


##### Selecting with Textual Column Expressions

When we construct a `Select` object using the `select()` function, we are normally passing to it _a series of `Table` and `Column` objects_ that were defined using table metadata, or when using the ORM we may be sending `ORM-mapped attributes` that represent _table columns_. However, sometimes there is also the need to _manufacture arbitrary SQL blocks inside of statements_, such as `constant string expressions`, or just some arbitrary SQL that's quicker to write literally.

The `text()` construct introduced at `Working with Transactions and the DBAPI` can in fact be _embedded into a `Select` construct directly_, such as below where we manufacture a hardcoded string literal `'some label'` and __embed__ it within the `SELECT` statement.

In [23]:
stmt = (
    select(
        text("'some phrase'"), user_table.c.name
    ).order_by(user_table.c.name)
)

with engine.connect() as conn:
    print(conn.execute(stmt).all())

2022-09-26 13:03:54,417 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-26 13:03:54,419 INFO sqlalchemy.engine.Engine SELECT 'some phrase', user_account.name 
FROM user_account ORDER BY user_account.name
2022-09-26 13:03:54,421 INFO sqlalchemy.engine.Engine [generated in 0.00408s] ()
[('some phrase', 'patrick'), ('some phrase', 'sandy'), ('some phrase', 'spongebob')]
2022-09-26 13:03:54,423 INFO sqlalchemy.engine.Engine ROLLBACK


While the `text()` construct can be used in most places to _inject literal SQL phrases_, more often than not we are actually dealing with textual units that each represent an individual column expression. In this common case we can get more functionality out of our textual fragment using the `literal_column()` construct instead. This object is similar to `text()` except that _instead of representing arbitrary SQL of any form, it **explicitly represents `a single column`** and can then be `labeled and referred` towards in `subqueries and other expressions`_.

In [24]:
stmt = (
    select(
        literal_column("'some phrase'").label("p"), user_table.c.name,
    ).order_by(user_table.c.name)
)

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.p}, {row.name}")

2022-09-26 13:03:54,531 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-26 13:03:54,533 INFO sqlalchemy.engine.Engine SELECT 'some phrase' AS p, user_account.name 
FROM user_account ORDER BY user_account.name
2022-09-26 13:03:54,534 INFO sqlalchemy.engine.Engine [generated in 0.00267s] ()
some phrase, patrick
some phrase, sandy
some phrase, spongebob
2022-09-26 13:03:54,537 INFO sqlalchemy.engine.Engine ROLLBACK


Note that _in both cases_, when using `text()` or `literal_column()`, we are writing a **syntactical SQL expression**, and _not a literal value_. We therefore have to include whatever quoting or syntaxes are necessary for the SQL we want to see rendered.

#### The WHERE clause

SQLAlchemy allows us to _compose SQL expressions_, such as `name = 'squidward'` or `user_id > 10`, by making use of _standard `Python operators` in conjunction with `Column` and similar objects_. For `boolean expressions`, most Python operators such as `==, !=, <, >= etc.` _generate new SQL Expression objects, rather than plain boolean `True/False` values_.

In [25]:
print(user_table.c.name == "squidward")
print(address_table.c.user_id > 10)

user_account.name = :name_1
address.user_id > :user_id_1


We can use expressions like these to _generate the `WHERE` clause_ by passing the resulting objects to the `Select.where()` method.

In [26]:
print(select(user_table).where(user_table.c.name == "squidward"))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


To produce _multiple expressions `joined by AND`_, the `Select.where()` method may be _invoked any number of times_.

In [27]:
print(
    select(address_table.c.email_address).
    where(user_table.c.name == "squidward").
    where(address_table.c.user_id == user_table.c.id)
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


A single call to `Select.where()` also _accepts multiple expressions_ with the same effect.

In [28]:
print(
    select(address_table.c.email_address).
    where(
        user_table.c.name == "squidward",
        address_table.c.user_id == user_table.c.id
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


`AND` and `OR` conjunctions are both __available directly__ using the `and_()` and `or_()` functions, illustrated below in terms of _ORM entities_.

In [29]:
print(
    select(Address.email_address).
    where(
        and_(
            or_(User.name == "squidward", User.name == "sandy"),
            Address.user_id == User.id
        )
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE (user_account.name = :name_1 OR user_account.name = :name_2) AND address.user_id = user_account.id


For *simple `"equality"` comparisons against a __single entity__*, there's also a popular method known as `Select.filter_by()` which accepts _keyword arguments that match to column keys or ORM attribute names_. It will filter against the _leftmost `FROM` clause_ or the _last entity joined_.

In [30]:
print(select(User).filter_by(name="spongebob", fullname="Spongebob Squarepants"))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1 AND user_account.fullname = :fullname_1


#### Explicit FROM clauses and JOINs

As mentioned previously, the `FROM` clause is usually _**inferred** based on the expressions_ that we are setting in the `columns` clause as well as other elements of the `Select`.

If we set a _single column_ from a particular `Table` in the `COLUMNS` clause, it puts that `Table` in the `FROM` clause as well.

In [31]:
print(select(user_table.c.name))

SELECT user_account.name 
FROM user_account


If we were to put _columns from two tables_, then we get a `comma-separated FROM clause`.

In [32]:
print(select(user_table.c.name, address_table.c.email_address))

SELECT user_account.name, address.email_address 
FROM user_account, address


In order to `JOIN` these two tables together, we typically use one of two methods on `Select`. The first is the `Select.join_from()` method, which _allows us to indicate the **left and right** side of the JOIN **explicitly**_.

In [33]:
print(
    select(user_table.c.name, address_table.c.email_address).
    join_from(user_table, address_table)
)

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


The other is the the `Select.join()` method, which indicates only the **right side** of the `JOIN`, the _left hand-side is inferred_.

In [34]:
print(
    select(user_table.c.name, address_table.c.email_address).
    join(address_table)
)

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


> ###### The ON Clause is inferred
>
> When using `Select.join_from()` or `Select.join()`, we may observe that the `ON clause` of the join is also __inferred__ for us in _simple foreign key cases_.

We also have the _option to add elements to the `FROM` clause explicitly_, if it is not inferred the way we want from the columns clause. We use the `Select.select_from()` method to achieve this, as below where we establish `user_table` as the first element in the FROM clause and `Select.join()` to establish `address_table` as the second.

In [35]:
print(
    select(address_table.c.email_address).
    select_from(user_table).join(address_table)
)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


Another example where we might want to use `Select.select_from()` is if our columns clause _doesn't have enough information_ to provide for a `FROM` clause. For example, to `SELECT` from the common SQL expression `count(*)`, we use a SQLAlchemy element known as `sqlalchemy.sql.expression.func` to produce the SQL `count()` function.

In [36]:
print(select(func.count("*")).select_from(user_table))

SELECT count(:count_2) AS count_1 
FROM user_account


##### Setting the ON Clause

The previous examples of `JOIN` illustrated that the `Select` construct _can join between two tables and produce the ON clause automatically_. This occurs in those examples because _the `user_table` and `address_table` Table objects include **a single `ForeignKeyConstraint`** definition_ which is used to form this `ON` clause.

If the _`left and right targets of the join` do not have such a constraint_, or there are _multiple constraints_ in place, we __need to specify the `ON` clause directly__. Both `Select.join()` and `Select.join_from()` accept an additional argument for the `ON` clause, which is stated using the same SQL Expression mechanics as we saw about in The `WHERE` clause.

In [37]:
print(
    select(address_table.c.email_address).
    select_from(user_table).
    join(address_table, user_table.c.id == address_table.c.user_id)
)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


> **ORM Tip** - there's another way to generate the `ON` clause when _using ORM entities_ that make use of the `relationship()` construct, like the mapping set up in the previous section at `Declaring Mapped Classes`. This is a whole subject onto itself, which is introduced at length at Using Relationships to Join.

##### OUTER and FULL join

Both the `Select.join()` and `Select.join_from()` methods accept keyword arguments `Select.join.isouter` and `Select.join.full` which will render **`LEFT OUTER JOIN`** and **`FULL OUTER JOIN`**, respectively.

In [38]:
print(select(user_table).join(address_table, isouter=True))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account LEFT OUTER JOIN address ON user_account.id = address.user_id


In [39]:
print(select(user_table).join(address_table, full=True))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account FULL OUTER JOIN address ON user_account.id = address.user_id


There is also a method `Select.outerjoin()` that is equivalent to using `.join(..., isouter=True)`.

> `SQL` also has a **`RIGHT OUTER JOIN`**. __`SQLAlchemy` doesn't render this directly__; instead, __reverse the order of the tables and use `LEFT OUTER JOIN`__.

#### ORDER BY, GROUP BY, HAVING

The `SELECT SQL statement` includes a clause called `ORDER BY` which is used to _return the selected rows within a given ordering_.

The `GROUP BY` clause is constructed similarly to the `ORDER BY` clause, and has the purpose of _sub-dividing the selected rows into specific groups upon which aggregate functions_ may be invoked. _The `HAVING` clause is usually used with `GROUP BY`_ and is of a similar form to the `WHERE` clause, except that it's _applied to the aggregated functions used within groups_.

#### ORDER BY

The `ORDER BY` clause is constructed in terms of SQL Expression constructs typically based on `Column` or _similar objects_. The `Select.order_by()` method accepts _one or more of these expressions_ **positionally**.

In [40]:
print(select(user_table).order_by(user_table.c.name))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.name


`Ascending/descending` is available from the `ColumnElement.asc()` and `ColumnElement.desc()` modifiers, which are present from _ORM-bound attributes_ as well.

In [41]:
print(select(User).order_by(User.fullname.desc()))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.fullname DESC


The above statement will yield rows that are `sorted` by the `user_account.fullname` column in _descending order_.